# Example notebook showing pipeline for schizophrenia analysis

In [1]:
# quick/dirty method to allow importing
import os
os.chdir("../")
os.getcwd()

'/home/eerdem/CELLECT-revised'

## Imports

In [2]:
import pandas as pd
import constants
from scripts import convert_output_to_dataframe
from scripts import gene_set_enrichment_analysis
from scripts import calculate_beta_correlation
from scripts import calculate_es_correlation

## Correct pvalue

In [3]:
# type of pval correction (can be modified in the constants.py file)
print(constants.PVAL_CORRECTION)

bonferroni


In [4]:
# After running CELLECT with the GWAS and scRNA files the output
# will be stored in out/CELLECT-{GWAS_name} folder
cellect_out_dir = "out/CELLECT-SCZ_PGC3_2020"
gwas_name = 'SCZ_PGC3_2020'

df_scz = convert_output_to_dataframe.make_df(cellect_out_dir)
df_scz = convert_output_to_dataframe.pvalue_correction(df_scz, method=constants.PVAL_CORRECTION)
display(df_scz)

,gwas,specificity_id,annotation,beta,beta_se,pvalue,method,n_methods,pvalue_bonferroni
0,SCZ_PGC3_2020,Allen_human_LGN,GABAergic.LGN_Inh_CTXN3,-0.017441,0.122443,0.556633,H-MAGMA,3,1.0
1,SCZ_PGC3_2020,Allen_human_LGN,GABAergic.LGN_Inh_LAMP5,-0.129795,0.098972,0.905128,H-MAGMA,3,1.0
2,SCZ_PGC3_2020,Allen_human_LGN,GABAergic.LGN_Inh_NTRK1,0.052677,0.121780,0.332673,H-MAGMA,3,1.0
3,SCZ_PGC3_2020,Allen_human_LGN,GABAergic.LGN_Inh_TRPC4,-0.308010,0.136322,0.988061,H-MAGMA,3,1.0
4,SCZ_PGC3_2020,Allen_human_LGN,Glutamatergic.LGN_Exc_BTNL9,-0.126593,0.182346,0.756227,H-MAGMA,3,1.0
...,...,...,...,...,...,...,...,...,...
1576,SCZ_PGC3_2020,tabula_muris,Tongue.keratinocyte,-0.178351,0.069668,0.994761,MAGMA,3,1.0
1577,SCZ_PGC3_2020,tabula_muris,Trachea.blood_cell,-0.288160,0.090981,0.999229,MAGMA,3,1.0
1578,SCZ_PGC3_2020,tabula_muris,Trachea.endothelial_cell,-0.117255,0.085372,0.915182,MAGMA,3,1.0
1579,SCZ_PGC3_2020,tabula_muris,Trachea.epithelial_cell,-0.346570,0.080956,0.999991,MAGMA,3,1.0


In [78]:
# show significant cell-types
value_counts = df_scz[
    (df_scz[f'pvalue_{constants.PVAL_CORRECTION}']<=0.05)
].value_counts(['specificity_id','annotation'])
significant_celltypes = value_counts[value_counts==3].index.tolist() # significant in all 3 methods
significant_celltypes = [f'{d}, {c}' for d,c in significant_celltypes]
display(significant_celltypes)

['mousebrain, TEINH11',
 'mousebrain, TEGLU18',
 'mousebrain, HBGLU7',
 'mousebrain, TEINH13',
 'mousebrain, TEINH12',
 'mousebrain, TEGLU15',
 'mousebrain, MEINH2',
 'mousebrain, TEINH1',
 'mousebrain, TEGLU9',
 'mousebrain, TEGLU8',
 'mousebrain, TEGLU7',
 'mousebrain, HBINH5',
 'mousebrain, TEGLU5',
 'mousebrain, TEGLU4',
 'mousebrain, TEGLU3',
 'mousebrain, TEGLU24',
 'mousebrain, TEGLU23',
 'mousebrain, MEGLU6',
 'mousebrain, TEGLU17',
 'mousebrain, TEGLU20',
 'mousebrain, TEGLU2',
 'mousebrain, HBGLU6',
 'mousebrain, TEINH16',
 'mousebrain, HBGLU5',
 'mousebrain, TEINH18',
 'Allen_human_MTG, Glutamatergic.Exc_L2_LAMP5_LTK',
 'DroNc_Human_Hippocampus, exCA1',
 'mousebrain, TEGLU10',
 'DroNc_Human_Hippocampus, exPFC1',
 'DroNc_Human_Hippocampus, exPFC2',
 'PsychENCODE_DER-22, Ex1',
 'tabula_muris, Brain_Non-Myeloid.oligodendrocyte_precursor_cell',
 'mousebrain, TEGLU11',
 'tabula_muris, Brain_Non-Myeloid.neuron',
 'mousebrain, DEGLU1',
 'mousebrain, DEGLU2',
 'mousebrain, TEGLU13',

## Gene-Set Enrichment Analysis

In [5]:
# gene-sets used in the analysis (can be modified in the constants.py file)
for gene_set in constants.GENE_SET_LIST:
    print(gene_set)

ARCHS4_Tissues
Aging_Perturbations_from_GEO_down
Aging_Perturbations_from_GEO_up
Allen_Brain_Atlas_down
Allen_Brain_Atlas_up
ClinVar_2019
Disease_Signatures_from_GEO_down_2014
Disease_Signatures_from_GEO_up_2014
ENCODE_Histone_Modifications_2013
ENCODE_Histone_Modifications_2015
Epigenomics_Roadmap_HM_ChIP-seq
GO_Biological_Process_2018
GO_Cellular_Component_2018
GO_Molecular_Function_2018
GTEx_Tissue_Sample_Gene_Expression_Profiles_down
GTEx_Tissue_Sample_Gene_Expression_Profiles_up
GWAS_Catalog_2019
HMDB_Metabolites
Human_Gene_Atlas
Jensen_COMPARTMENTS
Jensen_DISEASES
Jensen_TISSUES
KEGG_2019_Human
Reactome_2016
Tissue_Protein_Expression_from_ProteomicsDB
WikiPathways_2019_Human
miRTarBase_2017


In [6]:
gwas_group_dict = {'schizophrenia':['SCZ']} # GWAS group name : [(regex) keywords]
# the GWAS group name is just used for printing to stdout and can be anything
# the keywords are used to find all gwas in the dataframe which contain the keywords
gsea_dict = gene_set_enrichment_analysis.gsea(df_scz, gwas_group_dict)
# since only one gwas (SCZ_PGC3_2020) is within the schizophrenia group all cell-types enriched have rank 1

Performing GSEA...

Top 5 ranked cell-types (N=68) in schizophrenia GWAS:
1. Allen_human_MTG: Glutamatergic.Exc_L2_LAMP5_LTK (N=1, freq=1.0)
1. mousebrain: TEGLU3 (N=1, freq=1.0)
1. mousebrain: TEINH1 (N=1, freq=1.0)
1. mousebrain: TEGLU9 (N=1, freq=1.0)
1. mousebrain: TEGLU8 (N=1, freq=1.0)
1. mousebrain: TEGLU7 (N=1, freq=1.0)
1. mousebrain: TEGLU5 (N=1, freq=1.0)
1. mousebrain: TEGLU4 (N=1, freq=1.0)
1. mousebrain: TEGLU24 (N=1, freq=1.0)
1. DroNc_Human_Hippocampus: GABA2 (N=1, freq=1.0)
1. mousebrain: TEGLU23 (N=1, freq=1.0)
1. mousebrain: TEGLU22 (N=1, freq=1.0)
1. mousebrain: TEGLU21 (N=1, freq=1.0)
1. mousebrain: TEGLU20 (N=1, freq=1.0)
1. mousebrain: TEGLU2 (N=1, freq=1.0)
1. mousebrain: TEGLU19 (N=1, freq=1.0)
1. mousebrain: TEINH11 (N=1, freq=1.0)
1. mousebrain: TEINH12 (N=1, freq=1.0)
1. mousebrain: TEINH13 (N=1, freq=1.0)
1. mousebrain: TEINH14 (N=1, freq=1.0)
1. mousebrain: TEINH15 (N=1, freq=1.0)
1. mousebrain: TEINH16 (N=1, freq=1.0)
1. mousebrain: TEINH17 (N=1, freq=1.0

Cell-type already analyzed. Skipping conversion...

(48/68) Converting mousebrain, DEGLU4 (151 genes)
Cell-type already analyzed. Skipping conversion...

(49/68) Converting mousebrain, DEINH1 (151 genes)
Cell-type already analyzed. Skipping conversion...

(50/68) Converting mousebrain, DEINH3 (151 genes)
Cell-type already analyzed. Skipping conversion...

(51/68) Converting mousebrain, HBGLU5 (151 genes)
Cell-type already analyzed. Skipping conversion...

(52/68) Converting mousebrain, HBGLU7 (151 genes)
Cell-type already analyzed. Skipping conversion...

(53/68) Converting mousebrain, TEGLU14 (151 genes)
Cell-type already analyzed. Skipping conversion...

(54/68) Converting mousebrain, HBGLU9 (151 genes)
Cell-type already analyzed. Skipping conversion...

(55/68) Converting mousebrain, HBINH5 (151 genes)
Cell-type already analyzed. Skipping conversion...

(56/68) Converting mousebrain, MEGLU6 (151 genes)
Cell-type already analyzed. Skipping conversion...

(57/68) Converting mousebrain

Cell-type already analyzed. Skipping analysis...

(56/68) Analyzing mousebrain, MEGLU6 (151 genes)...
Cell-type already analyzed. Skipping analysis...

(57/68) Analyzing mousebrain, MEINH10 (151 genes)...
Cell-type already analyzed. Skipping analysis...

(58/68) Analyzing mousebrain, MEINH2 (151 genes)...
Cell-type already analyzed. Skipping analysis...

(59/68) Analyzing mousebrain, MSN1 (151 genes)...
Cell-type already analyzed. Skipping analysis...

(60/68) Analyzing mousebrain, MSN2 (151 genes)...
Cell-type already analyzed. Skipping analysis...

(61/68) Analyzing mousebrain, MSN3 (151 genes)...
Cell-type already analyzed. Skipping analysis...

(62/68) Analyzing mousebrain, MSN5 (151 genes)...
Cell-type already analyzed. Skipping analysis...

(63/68) Analyzing mousebrain, TEGLU1 (151 genes)...
Cell-type already analyzed. Skipping analysis...

(64/68) Analyzing mousebrain, TEGLU10 (151 genes)...
Cell-type already analyzed. Skipping analysis...

(65/68) Analyzing mousebrain, TEGLU11 

## Cell-type Correlation

In [7]:
# at least two different gwas should be analysed prior to calculating the correlation
# for this example a dataframe will be loaded including 500+ unqiue gwas

df_all = pd.read_hdf('data/CELLECT_output/data.h5', 'df_all')

display(df_all)
print(f"Number of unqiue GWAS: {df_all['gwas'].nunique()}")

,gwas,specificity_id,annotation,beta,beta_se,pvalue,method,n_methods,pvalue_bonferroni
0,AD_JANSENS2019,Allen_human_LGN,GABAergic.LGN_Inh_CTXN3,-0.174645,0.089013,0.975106,H-MAGMA,3,1.0
1,AD_JANSENS2019,Allen_human_LGN,GABAergic.LGN_Inh_LAMP5,-0.123540,0.071732,0.957472,H-MAGMA,3,1.0
2,AD_JANSENS2019,Allen_human_LGN,GABAergic.LGN_Inh_NTRK1,-0.269440,0.088436,0.998840,H-MAGMA,3,1.0
3,AD_JANSENS2019,Allen_human_LGN,GABAergic.LGN_Inh_TRPC4,-0.246886,0.099982,0.993223,H-MAGMA,3,1.0
4,AD_JANSENS2019,Allen_human_LGN,Glutamatergic.LGN_Exc_BTNL9,-0.421298,0.133637,0.999188,H-MAGMA,3,1.0
...,...,...,...,...,...,...,...,...,...
879625,volume: thalamus,tabula_muris,Tongue.keratinocyte,-0.035636,0.045386,0.783817,MAGMA,3,1.0
879626,volume: thalamus,tabula_muris,Trachea.blood_cell,-0.102483,0.059198,0.958279,MAGMA,3,1.0
879627,volume: thalamus,tabula_muris,Trachea.endothelial_cell,-0.114722,0.055573,0.980498,MAGMA,3,1.0
879628,volume: thalamus,tabula_muris,Trachea.epithelial_cell,-0.102770,0.052743,0.974312,MAGMA,3,1.0


Number of unqiue GWAS: 538


In [8]:
# corr_df = calculate_beta_correlation.calculate_celltype_corr(df_all) # calculates correlation
corr_df = pd.read_hdf('data/CELLECT_output/data.h5', key='corr_df') # or load pre-calculated corr

In [9]:
display(corr_df)

,gwasx,gwasy,corr,pval,method
0,AD_JANSENS2019,ASD_2019,-0.220065,2.278355e-07,H-MAGMA
1,AD_JANSENS2019,BIP_PGC3,-0.122407,4.318468e-03,H-MAGMA
2,AD_JANSENS2019,BMI_GIANT2018,-0.007513,8.614700e-01,H-MAGMA
3,AD_JANSENS2019,DTI AD: ACR,0.247713,5.065423e-09,H-MAGMA
4,AD_JANSENS2019,DTI AD: ALIC,0.234532,3.299649e-08,H-MAGMA
...,...,...,...,...,...
144448,volume: caudate,volume: putamen,0.337825,6.184427e-16,MAGMA
144449,volume: caudate,volume: thalamus,0.351890,3.040520e-17,MAGMA
144450,volume: pallidum,volume: putamen,0.256756,1.313908e-09,MAGMA
144451,volume: pallidum,volume: thalamus,0.310798,1.330849e-13,MAGMA


In [10]:
corr_df[
    # get only correlations including scz
    ((corr_df['gwasx']==gwas_name)|(corr_df['gwasy']==gwas_name))
    &
    # get only significant correlations
    (corr_df['pval']<calculate_beta_correlation.get_pthres(corr_df))
].pivot(index='method',columns=['gwasx','gwasy'])['corr'].T # where NaN implies not significant correlation

,method,H-MAGMA,LDSC,MAGMA
gwasx,gwasy,,,
ASD_2019,SCZ_PGC3_2020,0.523282,NaN,0.603968
BIP_PGC3,SCZ_PGC3_2020,0.870584,0.851139,0.889868
BMI_GIANT2018,SCZ_PGC3_2020,0.845410,0.710721,0.817630
DTI FA: CGC,SCZ_PGC3_2020,0.528416,NaN,NaN
DTI FA: IFO,SCZ_PGC3_2020,0.539209,0.528535,NaN
DTI MO: CGC,SCZ_PGC3_2020,0.551671,NaN,NaN
DTI PC1: IFO,SCZ_PGC3_2020,0.534203,0.534127,NaN
InstantCoffee_2019,SCZ_PGC3_2020,0.535598,0.740112,0.689086
PGC_depression2019,SCZ_PGC3_2020,0.726869,0.739430,0.787090


## ES Gene Correlation

In [60]:
datasets = df_all['specificity_id'].unique()
print(datasets)

['Allen_human_LGN' 'Allen_human_MTG' 'Descartes_Human_Cerebellum'
 'Descartes_Human_Cerebrum' 'DroNc_Human_Hippocampus'
 'GSE101601_Temporal_Cortex' 'GSE67835_Human_Cortex' 'PsychENCODE_DER-22'
 'mousebrain' 'tabula_muris']


In [12]:
# es_corr_df = calculate_es_correlation.calculate_es_corr(datasets) # calculates correlation
es_corr_df = pd.read_hdf('data/CELLECT_output/data.h5', key='es_corr_df') # or load pre-calculated corr

In [13]:
display(es_corr_df)

,celltypex,celltypey,corr,pval,pval_bonferroni
0,"Allen_human_LGN, GABAergic.LGN_Inh_CTXN3","Allen_human_LGN, GABAergic.LGN_Inh_LAMP5",0.155456,3.503827e-15,5.194074e-10
1,"Allen_human_LGN, GABAergic.LGN_Inh_CTXN3","Allen_human_LGN, GABAergic.LGN_Inh_NTRK1",0.360667,1.160120e-115,1.719762e-110
2,"Allen_human_LGN, GABAergic.LGN_Inh_CTXN3","Allen_human_LGN, GABAergic.LGN_Inh_TRPC4",0.209286,6.473715e-26,9.596634e-21
3,"Allen_human_LGN, GABAergic.LGN_Inh_CTXN3","Allen_human_LGN, Glutamatergic.LGN_Exc_BTNL9",0.159302,1.347408e-09,1.997397e-04
4,"Allen_human_LGN, GABAergic.LGN_Inh_CTXN3","Allen_human_LGN, Glutamatergic.LGN_Exc_PRKCG_BCHE",0.246957,1.938275e-24,2.873299e-19
...,...,...,...,...,...
148235,"tabula_muris, Trachea.blood_cell","tabula_muris, Trachea.epithelial_cell",0.079243,1.409471e-02,1.000000e+00
148236,"tabula_muris, Trachea.blood_cell","tabula_muris, Trachea.mesenchymal_cell",0.111152,1.375545e-07,2.039108e-02
148237,"tabula_muris, Trachea.endothelial_cell","tabula_muris, Trachea.epithelial_cell",0.089872,9.832454e-04,1.000000e+00
148238,"tabula_muris, Trachea.endothelial_cell","tabula_muris, Trachea.mesenchymal_cell",0.149082,1.478933e-18,2.192370e-13


In [61]:
es_corr_df[
    (
        # get only celltypes significant in scz
        (es_corr_df['celltypex'].isin(significant_celltypes))
        |
        (es_corr_df['celltypey'].isin(significant_celltypes))
    )
    &
    # get only significant correlations
    (es_corr_df['pval_bonferroni']<=0.05)
].sort_values('corr', ascending=False)

,celltypex,celltypey,corr,pval,pval_bonferroni
100035,"mousebrain, HBGLU6","mousebrain, HBGLU7",0.613823,0.000000e+00,0.000000
138510,"mousebrain, TEINH11","mousebrain, TEINH12",0.570274,0.000000e+00,0.000000
86202,"mousebrain, DEINH3","mousebrain, MEINH2",0.564248,0.000000e+00,0.000000
134888,"mousebrain, TEGLU11","mousebrain, TEGLU3",0.557421,0.000000e+00,0.000000
134710,"mousebrain, TEGLU10","mousebrain, TEGLU11",0.539221,0.000000e+00,0.000000
...,...,...,...,...,...
134215,"mousebrain, SZNBL","mousebrain, TEGLU11",-0.092368,3.846871e-08,0.005703
41323,"Allen_human_MTG, Non-neuronal.Astro_L1-6_FGFR3...","DroNc_Human_Hippocampus, exPFC2",-0.107081,2.990086e-08,0.004433
57758,"DroNc_Human_Hippocampus, exPFC2","PsychENCODE_DER-22, Oligo",-0.118070,4.679568e-08,0.006937
40861,"Allen_human_MTG, Non-neuronal.Astro_L1-2_FGFR3...","DroNc_Human_Hippocampus, exPFC2",-0.119805,2.386280e-07,0.035374
